<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/Examples/How_to_solve_a_classical_electrodynamics_problem_using_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to solve a classical electrodynamics problem using python

## Example: Griffiths (5th) Problem 2.12

### Using Gauss's Law to Find the Electric Field of a Spherical Shell
#### Problem
We want to find the electric field $\mathbf{E}$ both inside and outside a spherical shell of radius $R$
that carries a uniform surface charge density $\sigma$, using Gauss's law.
#### Key Concepts
##### Gauss's Law
Gauss's law states:
$$
\oint \mathbf{E} \cdot d\mathbf{A} = \frac{Q_{\text{enc}}}{\varepsilon_0},
$$
where:
- $\mathbf{E}$: Electric field,
- $d\mathbf{A}$: Differential area vector,
- $Q_{\text{enc}}$: Enclosed charge,
- $\varepsilon_0$: Permittivity of free space.

##### Symmetry of the Problem
- The spherical symmetry of the shell ensures that the electric field, if present, will be radial and constant over a spherical Gaussian surface.
- Depending on whether we are inside or outside the shell, the enclosed charge $Q_{\text{enc}}$ will change.

---

##### Steps:
1. **Inside the Shell** $( r < R )$: No charge is enclosed within the Gaussian surface, so $\mathbf{E} = 0$.
2. **Outside the Shell** $( r \geq R )$: The entire charge $ Q = 4\pi R^2 \sigma $ is enclosed, and the field is calculated using Gauss's law.

In [ ]:
# Constants and Imports
import sympy as sp

# Define symbols
r, R, sigma, epsilon_0 = sp.symbols("r R sigma epsilon_0", positive=True)


#### Inside the Shell (r < R)

For a spherical shell, the enclosed charge $ Q_{\text{enc}} $ inside the shell is zero because all the charge resides on the surface.
Using Gauss's law:
$$
\oint \mathbf{E} \cdot d\mathbf{A} = \frac{Q_{\text{enc}}}{\varepsilon_0},
$$
and since $ Q_{\text{enc}} = 0 $, we have:
$$
\mathbf{E} = 0 \quad \text{for } r < R.
$$


In [ ]:
E_inside = 0
E_inside


0

##### Outside the Shell $( r \geq R )$
For $ r \geq R $, the entire charge $ Q $ is enclosed within the Gaussian surface.
- Surface area of the spherical Gaussian surface: $ A = 4\pi r^2 $,
- Total charge on the shell: $ Q = 4\pi R^2 \sigma $.

Using Gauss's law:
$$
\oint \mathbf{E} \cdot d\mathbf{A} = \frac{Q}{\varepsilon_0}.
$$

Substitute:
$$
E \cdot (4\pi r^2) = \frac{4\pi R^2 \sigma}{\varepsilon_0}.
$$

Solve for $ E $:
$$
E = \frac{R^2 \sigma}{\varepsilon_0 r^2} \quad \text{for } r \geq R.
$$

In [ ]:
Q = 4 * sp.pi * R**2 * sigma  # Total charge
A = 4 * sp.pi * r**2  # Surface area of Gaussian surface
E_outside = Q / (epsilon_0 * A)
E_outside.simplify()


R**2*sigma/(epsilon_0*r**2)

### Summary of Results

- **Inside the shell** $( r < R )$: $ \mathbf{E} = 0 $,
- **Outside the shell** $( r \geq R )$: $ \mathbf{E} = \frac{R^2 \sigma}{\varepsilon_0 r^2} $.



In [ ]:
# Display results
E_inside, E_outside.simplify()

(0, R**2*sigma/(epsilon_0*r**2))

To visualize we can use blender with the following script `electric_field_visualization.py` and execute:

```blender --background --python electric_field_visualization.py```

In [ ]:
# blender script for visualization
import bpy
import math
import mathutils

# Parameters
sigma = 1.0  # Surface charge density (arbitrary units)
epsilon_0 = 1.0  # Permittivity of free space (arbitrary units)
R = 2.0  # Radius of the spherical shell
field_scale = 5.0  # Scale factor for field line length
arrow_count = 10  # Number of arrows along each axis
output_file = "./electric_field_render.png"  # Output file path

# Function to calculate the electric field
def electric_field(position, radius, charge_density, permittivity):
    r = position.length
    if r < radius:
        return mathutils.Vector((0, 0, 0))  # Inside the shell, E = 0
    else:
        E_magnitude = (radius**2 * charge_density) / (permittivity * r**2)
        return position.normalized() * E_magnitude

# Clear existing objects
bpy.ops.wm.read_factory_settings(use_empty=True)

# Create the sphere
bpy.ops.mesh.primitive_uv_sphere_add(radius=R, location=(0, 0, 0))
sphere = bpy.context.object
sphere.name = "Charged Sphere"

# Material for the sphere
material = bpy.data.materials.new(name="SphereMaterial")
material.use_nodes = True
sphere.data.materials.append(material)

# Generate field lines as arrows
for x in range(-arrow_count, arrow_count + 1):
    for y in range(-arrow_count, arrow_count + 1):
        for z in range(-arrow_count, arrow_count + 1):
            position = mathutils.Vector((x, y, z))
            if position.length > R:  # Avoid placing arrows inside the shell
                field = electric_field(position, R, sigma, epsilon_0)
                if field.length > 0:
                    # Scale the arrow length for better visualization
                    field_scaled = field.normalized() * field.length * field_scale

                    # Add arrow at this location
                    bpy.ops.object.empty_add(type='ARROWS', location=position)
                    arrow = bpy.context.object
                    arrow.name = f"FieldArrow_{x}_{y}_{z}"

                    # Point the arrow in the direction of the field
                    arrow.rotation_mode = 'QUATERNION'
                    arrow.rotation_quaternion = field_scaled.to_track_quat('Z', 'Y')

                    # Scale the arrow to represent field magnitude
                    arrow.scale = (0.1, 0.1, field.length * field_scale)

# Set up camera
bpy.ops.object.camera_add(location=(15, -15, 10))
camera = bpy.context.object
camera.name = "Camera"
bpy.context.scene.camera = camera
camera.rotation_euler = (math.radians(60), 0, math.radians(45))

# Set up light
bpy.ops.object.light_add(type='SUN', location=(10, -10, 10))
light = bpy.context.object
light.name = "SunLight"

# Set up rendering
bpy.context.scene.render.engine = "CYCLES"  # Use Cycles renderer
bpy.context.scene.cycles.samples = 128  # Set rendering quality
bpy.context.scene.render.image_settings.file_format = "PNG"
bpy.context.scene.render.filepath = output_file

# Render the scene
bpy.ops.render.render(write_still=True)